In [8]:
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv

# 📂 .env 파일 불러오기
load_dotenv()
REST_API_KEY = os.getenv('KAKAO_API_KEY')

# 📂 광진구 어린이 보호구역 파일 읽기 (cp949 인코딩 주의)
child_zone_path = '../../../data/gwangjin_child_zone_cp949.csv'
child_df = pd.read_csv(child_zone_path, encoding='cp949')

# 📍 위도/경도 저장할 리스트 준비
latitudes = []
longitudes = []

# 📍 카카오 주소 → 좌표 변환 함수
def get_lat_lon(address, api_key):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()["documents"]
        if result:
            return float(result[0]["y"]), float(result[0]["x"])  # 위도(y), 경도(x)
        else:
            return None, None
    else:
        return None, None

# 📍 도로명주소 하나씩 변환
for address in child_df['도로명주소']:
    lat, lon = get_lat_lon(address, REST_API_KEY)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(0.1)  # 카카오 API 과부하 방지 (0.1초 쉬기)

# 📍 결과 추가
child_df['위도'] = latitudes
child_df['경도'] = longitudes

# 📂 변환된 파일로 저장 (cp949로 저장, 엑셀 호환)
output_path = '../../../data/gwangjin_child_zone_with_coords.csv'
child_df.to_csv(output_path, index=False, encoding='cp949')

print(f"✅ 변환 완료! 위경도 추가된 파일 저장: {output_path}")


✅ 변환 완료! 위경도 추가된 파일 저장: ../../../data/gwangjin_child_zone_with_coords.csv


In [9]:
import pandas as pd
import numpy as np

# 📂 파일 경로 설정
child_zone_path = '../../../data/gwangjin_child_zone_with_coords.csv'
cctv_path = '../../../results/CCTV/proceeded_cctv_with_cluster.csv'

# 📂 파일 불러오기
child_df = pd.read_csv(child_zone_path, encoding='cp949')
cctv_df = pd.read_csv(cctv_path, encoding='utf-8')

# 📍 Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (미터 단위)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# 📍 어린이 보호구역 데이터에 CCTV 여부 추가할 컬럼 준비
child_df['근처_CCTV_여부'] = 'X'  # 기본은 '없음'

# 📍 하나씩 비교
for idx, child_row in child_df.iterrows():
    if np.isnan(child_row['위도']) or np.isnan(child_row['경도']):
        continue  # 위도/경도 없으면 건너뜀
    
    lat1 = child_row['위도']
    lon1 = child_row['경도']
    
    # CCTV들과 거리 계산
    distances = haversine(lat1, lon1, cctv_df['위도'], cctv_df['경도'])
    
    # 100m 이내에 하나라도 있으면 'O'로 표시
    if (distances <= 100).any():
        child_df.at[idx, '근처_CCTV_여부'] = 'O'

# 📂 CCTV 없는 어린이 보호구역만 추출
no_cctv_childzone_df = child_df[child_df['근처_CCTV_여부'] == 'X']

# 📂 결과 저장
no_cctv_childzone_df.to_csv('../../../results/no_cctv_childzone.csv', index=False, encoding='cp949')

print(f"✅ 분석 완료! CCTV 없는 어린이 보호구역 {len(no_cctv_childzone_df)}개 발견")
print("✅ 결과 파일 저장: '../../../results/no_cctv_childzone.csv'")


✅ 분석 완료! CCTV 없는 어린이 보호구역 0개 발견
✅ 결과 파일 저장: '../../../results/no_cctv_childzone.csv'


In [4]:
# 데이터 UTF-8로 수정

import pandas as pd
import numpy as np

# 📂 파일 경로 설정
child_zone_path = '../../../data/gwangjin_child_zone_with_coords.csv'
cctv_path = '../../../results/CCTV/proceeded_cctv_with_cluster.csv'

# 📂 파일 불러오기
child_df = pd.read_csv(child_zone_path, encoding='cp949')
cctv_df = pd.read_csv(cctv_path, encoding='utf-8')

# 📍 Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (미터 단위)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# 📍 결과 저장할 컬럼 추가
child_df['100m이내_CCTV개수'] = 0

# 📍 어린이 보호구역마다 CCTV 개수 세기
for idx, child_row in child_df.iterrows():
    if np.isnan(child_row['위도']) or np.isnan(child_row['경도']):
        continue  # 위도/경도 없으면 건너뜀
    
    lat1 = child_row['위도']
    lon1 = child_row['경도']
    
    # CCTV들과 거리 계산
    distances = haversine(lat1, lon1, cctv_df['위도'], cctv_df['경도'])
    
    # 100m 이내 CCTV 개수
    cctv_count = (distances <= 100).sum()
    
    # 결과 저장
    child_df.at[idx, '100m이내_CCTV개수'] = cctv_count

# 📂 결과 저장
child_df.to_csv('../../../results/gwangjin_child_zone_with_cctv_counts_utf8.csv', index=False, encoding='utf-8-sig')


print(f"✅ 분석 완료! 결과 파일 저장: {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '../../../data/gwangjin_child_zone_with_coords.csv'

In [1]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd

# 📂 한글 폰트 설정 (나눔고딕)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 깨짐 방지

# 📋 기본 통계
total_zones = len(child_df)
zones_with_cctv = (child_df['100m이내_CCTV개수'] > 0).sum()
zones_without_cctv = (child_df['100m이내_CCTV개수'] == 0).sum()
coverage_rate = (zones_with_cctv / total_zones) * 100

print(f"✅ 어린이 보호구역 총 개수: {total_zones}")
print(f"✅ CCTV 설치된 보호구역 수: {zones_with_cctv}")
print(f"✅ CCTV 없는 보호구역 수: {zones_without_cctv}")
print(f"✅ CCTV 커버율: {coverage_rate:.2f}%")

# 📊 1. 전체 CCTV 설치 개수 분포 히스토그램
plt.figure(figsize=(8,5))
child_df['100m이내_CCTV개수'].hist(bins=range(0, child_df['100m이내_CCTV개수'].max()+2, 1))
plt.title('100m 이내 CCTV 설치 개수 분포')
plt.xlabel('CCTV 개수')
plt.ylabel('어린이 보호구역 수')
plt.grid(True)
plt.tight_layout()
plt.savefig('../../../results/cctv_distribution_histogram.png')  # ✅ 그래프 저장
plt.show()

print("✅ CCTV 설치 개수 분포 그래프 저장 완료: '../../../results/cctv_distribution_histogram.png'")

# 📋 2. CCTV 설치 개수가 가장 적은 보호구역 Top 5 출력
cctv_least_zones = child_df[['도로명주소', '100m이내_CCTV개수']].drop_duplicates()
cctv_least_zones_sorted = cctv_least_zones.sort_values(by='100m이내_CCTV개수').head(5)

# 결과 출력
print("✅ CCTV 설치가 가장 적은 어린이 보호구역 Top 5:")
print(cctv_least_zones_sorted)


NameError: name 'child_df' is not defined

In [30]:
import pandas as pd
import numpy as np
import folium

# 📂 파일 경로
child_zone_path = '../../../data/gwangjin_child_zone_with_coords.csv'
cctv_path = '../../../results/CCTV/proceeded_cctv_with_cluster.csv'

# 📂 파일 불러오기
child_df = pd.read_csv(child_zone_path, encoding='cp949')
cctv_df = pd.read_csv(cctv_path, encoding='utf-8')

# 📍 Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (미터 단위)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# 📍 50m 이내 CCTV 개수 계산
child_df['50m이내_CCTV개수'] = 0

for idx, child_row in child_df.iterrows():
    if np.isnan(child_row['위도']) or np.isnan(child_row['경도']):
        continue
    
    lat1 = child_row['위도']
    lon1 = child_row['경도']
    
    distances = haversine(lat1, lon1, cctv_df['위도'], cctv_df['경도'])
    cctv_count = (distances <= 50).sum()
    child_df.at[idx, '50m이내_CCTV개수'] = cctv_count

# 📋 50m 이내 커버율 출력
total_zones = len(child_df)
zones_with_cctv_50m = (child_df['50m이내_CCTV개수'] > 0).sum()
coverage_rate_50m = (zones_with_cctv_50m / total_zones) * 100

print(f"✅ 50m 이내 CCTV 커버율: {coverage_rate_50m:.2f}% (총 {zones_with_cctv_50m}/{total_zones}개소)")

# 📍 50m 이내 CCTV 1개 이하 보호구역 지도 표시
map_center = [37.5384, 127.0826]  # 광진구 중심점
m = folium.Map(location=map_center, zoom_start=13)

low_cctv_zones = child_df[child_df['50m이내_CCTV개수'] <= 1]

for idx, row in low_50m_zones_unique.iterrows():   # ✅ 유니크한 데이터로 돌려야 함
    if pd.isna(row['위도']) or pd.isna(row['경도']):
        continue

    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['도로명주소']} (50m 이내 CCTV: {row['50m이내_CCTV개수']}개)",
        icon=folium.Icon(color='red')
    ).add_to(m)

# 📂 지도 저장
map_path = '../../../results/low_cctv_childzones_map.html'
m.save(map_path)

print(f"✅ 지도 저장 완료: {map_path}")


✅ 50m 이내 CCTV 커버율: 80.30% (총 53/66개소)
✅ 지도 저장 완료: ../../../results/low_cctv_childzones_map.html


In [2]:
# 50m 이내 CCTV 개수가 1개 이하인 보호구역만 추출
low_50m_zones = child_df[child_df['50m이내_CCTV개수'] <= 1]

# 중복된 보호구역 제거 (유니크한 보호구역만)
low_50m_zones_unique = low_50m_zones.drop_duplicates(subset=['도로명주소', '위도', '경도'])

# Folium 지도에 마커 찍기 (유니크한 보호구역만)
m = folium.Map(location=[37.5384, 127.0826], zoom_start=13)

for idx, row in low_50m_zones_unique.iterrows():   # 유니크한 데이터로 돌려야 함
    if pd.isna(row['위도']) or pd.isna(row['경도']):
        continue

    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['도로명주소']} (50m 이내 CCTV: {row['50m이내_CCTV개수']}개)",
        icon=folium.Icon(color='red')
    ).add_to(m)

# 지도 저장
map_path = '../../../results/low_cctv_childzones_map.html'
m.save(map_path)

print(f"✅ 지도 저장 완료: {map_path}")


NameError: name 'child_df' is not defined

In [27]:
# 📋 50m 이내 CCTV가 0개인 보호구역만 보기 (1개 미만)

no_cctv_zones = child_df[child_df['50m이내_CCTV개수'] < 1]
no_cctv_zones_unique = no_cctv_zones.drop_duplicates(subset=['도로명주소', '위도', '경도'])

print(f"✅ 50m 이내 CCTV 0개 고유 보호구역 수: {len(no_cctv_zones_unique)}개")
print(no_cctv_zones_unique[['도로명주소', '위도', '경도', '50m이내_CCTV개수']])



✅ 50m 이내 CCTV 0개 고유 보호구역 수: 4개
   도로명주소         위도          경도  50m이내_CCTV개수
13  중곡4동  37.559078  127.089447             0
14  중곡2동  37.560295  127.081526             0
16   군자동  37.555477  127.075351             0
17  중곡3동  37.564039  127.086556             0


In [23]:
# 중복 값이 존재해 유니크로 다시 저장

import numpy as np
import pandas as pd

# 📂 파일 읽기
child_df = pd.read_csv('../../../results/Child/gwangjin_child_zone_with_cctv_counts.csv', encoding='cp949')
cctv_df = pd.read_csv('../../../results/CCTV/proceeded_cctv_with_cluster.csv', encoding='utf-8')

# 📍 Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (m)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# 📋 50m 이내 CCTV 개수 계산
child_df['50m이내_CCTV개수'] = 0

for idx, child_row in child_df.iterrows():
    if np.isnan(child_row['위도']) or np.isnan(child_row['경도']):
        continue
    
    lat1 = child_row['위도']
    lon1 = child_row['경도']
    distances = haversine(lat1, lon1, cctv_df['위도'], cctv_df['경도'])
    cctv_count = (distances <= 50).sum()
    child_df.at[idx, '50m이내_CCTV개수'] = cctv_count

# 📋 50m 이내 CCTV 0개 보호구역만 뽑기
no_cctv_zones = child_df[child_df['50m이내_CCTV개수'] < 1]

# 📋 위도/경도로 중복 제거
no_cctv_zones_unique = no_cctv_zones.drop_duplicates(subset=['위도', '경도'])

# 📋 결과 출력
print(f"✅ 50m 이내 CCTV 완전 미설치 고유 보호구역 수: {len(no_cctv_zones_unique)}개")
print(no_cctv_zones_unique[['도로명주소', '위도', '경도', '50m이내_CCTV개수']])


✅ 50m 이내 CCTV 완전 미설치 고유 보호구역 수: 4개
   도로명주소         위도          경도  50m이내_CCTV개수
13  중곡4동  37.559078  127.089447             0
14  중곡2동  37.560295  127.081526             0
16   군자동  37.555477  127.075351             0
17  중곡3동  37.564039  127.086556             0


In [29]:
# 100m 이내 CCTV 평균 개수
avg_cctv_100m = child_df['100m이내_CCTV개수'].mean()

# 50m 이내 CCTV 평균 개수
avg_cctv_50m = child_df['50m이내_CCTV개수'].mean()
low_50m_zones = child_df[child_df['50m이내_CCTV개수'] < avg_cctv_50m]
low_50m_zones_unique = low_50m_zones.drop_duplicates(subset=['도로명주소', '위도', '경도'])

print(f"✅ 100m 이내 평균 CCTV 개수: {avg_cctv_100m:.2f}대")
print(f"✅ 50m 이내 평균 CCTV 개수: {avg_cctv_50m:.2f}대")

print(f"✅ 50m 이내 평균 이하 고유 보호구역 수: {len(low_50m_zones_unique)}개")
print(low_50m_zones_unique[['도로명주소', '위도', '경도', '50m이내_CCTV개수']])

✅ 100m 이내 평균 CCTV 개수: 3.95대
✅ 50m 이내 평균 CCTV 개수: 1.53대
✅ 50m 이내 평균 이하 고유 보호구역 수: 9개
   도로명주소         위도          경도  50m이내_CCTV개수
1    광장동  37.546857  127.103026             1
3   구의2동  37.547228  127.089940             1
5    화양동  37.546476  127.071368             1
8   자양2동  37.528844  127.084415             1
13  중곡4동  37.559078  127.089447             0
14  중곡2동  37.560295  127.081526             0
16   군자동  37.555477  127.075351             0
17  중곡3동  37.564039  127.086556             0
43  구의1동  37.542412  127.085675             1


In [25]:
import pandas as pd
import numpy as np

# 📂 파일 불러오기
child_df = pd.read_csv('../../../results/Child/gwangjin_child_zone_with_cctv_counts.csv', encoding='cp949')
cctv_df = pd.read_csv('../../../results/CCTV/proceeded_cctv_with_cluster.csv', encoding='utf-8')

# 📍 Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (m)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# 📋 50m 이내 CCTV 개수 계산
child_df['50m이내_CCTV개수'] = 0

for idx, child_row in child_df.iterrows():
    if np.isnan(child_row['위도']) or np.isnan(child_row['경도']):
        continue
    
    lat1 = child_row['위도']
    lon1 = child_row['경도']
    distances = haversine(lat1, lon1, cctv_df['위도'], cctv_df['경도'])
    cctv_count = (distances <= 50).sum()
    child_df.at[idx, '50m이내_CCTV개수'] = cctv_count

# 📋 100m 기준 통계
avg_cctv_100m = child_df['100m이내_CCTV개수'].mean()
min_cctv_100m = child_df['100m이내_CCTV개수'].min()
max_cctv_100m = child_df['100m이내_CCTV개수'].max()
std_cctv_100m = child_df['100m이내_CCTV개수'].std()

# 📋 50m 기준 통계
avg_cctv_50m = child_df['50m이내_CCTV개수'].mean()
min_cctv_50m = child_df['50m이내_CCTV개수'].min()
max_cctv_50m = child_df['50m이내_CCTV개수'].max()
std_cctv_50m = child_df['50m이내_CCTV개수'].std()

# 📋 평균 이하 구역 (50m 기준)
low_50m_zones = child_df[child_df['50m이내_CCTV개수'] < avg_cctv_50m]

# ✅ 결과 출력
print("✅ [100m 기준]")
print(f"- 평균 CCTV 개수: {avg_cctv_100m:.2f}대")
print(f"- 최소 CCTV 개수: {min_cctv_100m}대")
print(f"- 최대 CCTV 개수: {max_cctv_100m}대")
print(f"- 표준편차: {std_cctv_100m:.2f}")

print("\n✅ [50m 기준]")
print(f"- 평균 CCTV 개수: {avg_cctv_50m:.2f}대")
print(f"- 최소 CCTV 개수: {min_cctv_50m}대")
print(f"- 최대 CCTV 개수: {max_cctv_50m}대")
print(f"- 표준편차: {std_cctv_50m:.2f}")

print(f"\n✅ 50m 이내 평균 이하 구역 수: {len(low_50m_zones)}개")
print(low_50m_zones[['도로명주소', '위도', '경도', '50m이내_CCTV개수']])


✅  기준]
- 평균 CCTV 개수: 3.95대
- 최소 CCTV 개수: 2대
- 최대 CCTV 개수: 6대
- 표준편차: 1.23

✅  기준]
- 평균 CCTV 개수: 1.53대
- 최소 CCTV 개수: 0대
- 최대 CCTV 개수: 4대
- 표준편차: 1.28

✅ 50m 이내 평균 이하 구역 수: 43개
   도로명주소         위도          경도  50m이내_CCTV개수
1    광장동  37.546857  127.103026             1
2    광장동  37.546857  127.103026             1
3   구의2동  37.547228  127.089940             1
5    화양동  37.546476  127.071368             1
6   구의2동  37.547228  127.089940             1
8   자양2동  37.528844  127.084415             1
10  자양2동  37.528844  127.084415             1
11  자양2동  37.528844  127.084415             1
12   광장동  37.546857  127.103026             1
13  중곡4동  37.559078  127.089447             0
14  중곡2동  37.560295  127.081526             0
16   군자동  37.555477  127.075351             0
17  중곡3동  37.564039  127.086556             0
18  중곡3동  37.564039  127.086556             0
19  자양2동  37.528844  127.084415             1
20   군자동  37.555477  127.075351             0
21   광장동  37.546857  127.103026            